In [1]:
import datetime
import sys
sys.path.append("..")
import random
from numpy.random import default_rng
from annoy import AnnoyIndex
import torch.autograd as autograd
from typing import List
import anndata
from functools import partial
from copy import deepcopy

from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from sklearn.metrics import (adjusted_rand_score, calinski_harabasz_score,
                             normalized_mutual_info_score, silhouette_score)
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import preprocessing

import utils
from sklearn import metrics
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.manifold import TSNE

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from random import sample
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import scanpy as sc
import time
import os
from scipy import sparse

from utils.explanation_utils import explanation_hook, get_explanation

torch.cuda.set_device(2)

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True

plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
path= "../"

# check available files
# !ls ../real_data

torch.autograd.set_detect_anomaly(True)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
adata = sc.read_h5ad("data/pbmc.h5ad")

# storing the layers count
adata.layers["counts"] = adata.X

In [3]:
# flavor: seurat_v3, cell_ranger & log = false
def sub_data_preprocess(adata: sc.AnnData, n_top_genes: int = 5000, batch_key: str = None, flavor: str = 'seurat_v3', min_genes: int = 200, min_cells: int = 3) -> sc.AnnData:
    sc.pp.filter_cells(adata, min_genes=min_genes)
    sc.pp.filter_genes(adata, min_cells=min_cells)
    if flavor == 'seurat_v3':
# #         count data is expected when flavor=='seurat_v3'
#         sc.pp.highly_variable_genes(
#             adata, flavor=flavor, batch_key = batch_key)
        sc.pp.highly_variable_genes(
            adata, flavor=flavor, batch_key = batch_key, n_top_genes=n_top_genes)

#     if flavor != 'seurat_v3':.
#         # log-format data is expected when flavor!='seurat_v3'
#         sc.pp.highly_variable_genes(
#             adata, n_top_genes=n_top_genes, flavor=flavor)
    sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
    sc.pp.log1p(adata)
    return adata


def data_preprocess(adata: sc.AnnData, key: str = 'batch', n_top_genes: int = 10000, flavor: str = 'seurat_v3', min_genes: int = 200, min_cells: int = 3, n_batch: int = 2) -> sc.AnnData:
    print('Establishing Adata for Next Step...')
    hv_adata = sub_data_preprocess(adata, n_top_genes=n_top_genes, batch_key = key, flavor=flavor, min_genes=min_genes, min_cells=min_cells)
    if len(adata.var.index) > n_top_genes:
        hv_adata = hv_adata[:, hv_adata.var['highly_variable']]
        
#     hv_adata.X = np.expm1(hv_adata.X)
    print('PreProcess Done.')
    return hv_adata

In [4]:
batch_str = "batch"

adata = data_preprocess(adata, batch_str)
adata  # Output the basic information of the preprocessed data.

Establishing Adata for Next Step...
PreProcess Done.


View of AnnData object with n_obs × n_vars = 15476 × 10000
    obs: 'Sample', 'n_counts', 'n_genes', 'batch', 'louvain', 'anno', 'Method', 'CellType'
    var: 'gene_name', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'hvg', 'log1p'
    layers: 'counts'

In [5]:
# # apply the pre processing onto the anndata
# # first create layer for the anndata
# adata.layers["X_raw"] = adata.X.copy()
# adata.layers["X_scaled"] = adata.X.copy()

# # split per batch into new objects.
# batches = list(set(adata.obs['batch']))
# # alldata_2 = {}

# # apply scaling to each batch in the pre process raw data
# for batch in batches:
# #     alldata[batch] = adata_int[adata_int.obs['Batch'] == batch,]

#     batch_index = adata.obs['batch'] == batch
#     temp_X = adata[batch_index].X
    
#     # log normalise data the same seurat
# #     sc.pp.normalize_per_cell(temp_adata, counts_per_cell_after = 1e4)
# #     sc.pp.log1p(temp_adata)
# #     temp_adata.layers["X_norm"] = temp_adata.X
    
#     # scaling of the data for PCA and UMAP embedding
#     temp_X_scaled = sc.pp.scale(temp_X, copy = True)
#     adata[batch_index].layers["X_scaled"] = temp_X_scaled

# # use this pancreatic data since it has been normalised already based on the author
# # adata.X = adata.layers["X_raw"].copy() # change to norm for count dataset

# # this is use for UMAP generation only for the raw dataset
# # adata.X = adata.layers["X_scaled"]

In [6]:
adata.obs.to_csv("metadata_cell_pbmc_full.csv", index = False)
adata.var.to_csv("metadata_gene_pbmc_full.csv", index = False)

In [7]:
pd.DataFrame(adata.X.transpose()).to_csv("count_matrix_pbmc_full.csv", index = False)